# Convolutional Neural Networks
---
In this notebook, I'll train a **CNN** to classify the sentiment of movie reviews in a corpus of text. The approach will be as follows:
* Pre-process movie reviews and their corresponding sentiment labels (positive = 1, negative = 0).
* Load in a **pre-trained** Word2Vec model, and use it to tokenize the reviews.
* Create training/validation/test sets of data.
* Define a `SentimentCNN` model that has a pre-trained embedding layer, convolutional layers, and a final, fully-connected, classification layer.
* Train and evaluate the model.

An example of a positive and negative review are shown below.

<img src='notebook_ims/reviews_ex.png' width=30% height=70% />

The task of text classification has typically been done with an RNN, which accepts a sequence of words as input and has a hidden state that is dependent on that sequence and acts as a kind of memory. You can see an example that classifies this same review dataset using an RNN in [this Github repository](https://github.com/udacity/deep-learning-v2-pytorch/blob/master/sentiment-rnn/Sentiment_RNN_Solution.ipynb). 


## Resources

This example shows how you can utilize convolutional layers to find patterns in sequences of word embeddings and create an effective text classifier using a CNN-based approach.

**1. Original paper**
* The code follows the structure outlined in the paper, [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882) by Yoon Kim (2014). 

**2. Pre-trained Word2Vec model**

* The key to this approach is convolving over word embeddings, for which I will use a pre-trained [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) model. 
* I am specifically using a "slim"-version of a model that was trained on part of a Google News dataset (about 100 billion words). The [original model](https://code.google.com/archive/p/word2vec/) contains 300-dimensional vectors for 3 million words and phrases.
* The "slim" model is cut to 300k English words, as described in [this Github repository](https://github.com/eyaler/word2vec-slim).

You should be able to modify this code slightly to make it compatible with a Word2Vec model of your choosing.

**3. Movie reviews data **

The dataset holds 25000 movie reviews, which were obtained from the movie review site, IMDb.


---
## Load in and Visualize the Data

In [7]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [18]:
import json
with open("../preprocess/arc_split.json") as f:
    data = json.load(f)



<function dict.keys>

In [177]:
import json
with open("../preprocess/nu_split.json") as f:
    nu_data = json.load(f)


In [178]:
nu_data['train'].values()

dict_values([{'text': 'create a storyboard for a sequel to your book use the same characters', 'label': 2}, {'text': 'briefly describe the authors approach to the liabilities of the goto statement', 'label': 0}, {'text': 'specify a deterministic finite automaton dfa recognizing the', 'label': 1}, {'text': 'how to construct the optimal maintenance problem', 'label': 1}, {'text': 'specify the thickness of a galvanized coating based on atmospheric data level ', 'label': 1}, {'text': 'discuss the nature of socialism', 'label': 0}, {'text': 'what is the title of the first movie you saw ', 'label': 0}, {'text': 'how many years does this animal live', 'label': 0}, {'text': 'when did the first saudi radio station open', 'label': 0}, {'text': 'contrast olympic athletes of today with athletes of past olympic games', 'label': 2}, {'text': 'express the following percentages as fractions in simplest form', 'label': 0}, {'text': 'compare how different children come to school', 'label': 2}, {'text': 

In [ ]:
import json
with open("../preprocess/arg_split.json") as f:
    arg_data = json.load(f)

arg_tokenized_text = tokenize_all_text(embed_lookup, arg_data['train'])
arg_test_tokenized_text = tokenize_all_text(embed_lookup, arg_data['test'])

# Test your implementation!

seq_length = 15

arg_train_features = pad_features(arg_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(arg_train_features)==len(arg_tokenized_text), "Features should have as many rows as reviews."
assert len(arg_train_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(arg_train_features[:20,:8])

# Test your implementation!

arg_test_features = pad_features(arg_test_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(arg_test_features)==len(arg_test_tokenized_text), "Features should have as many rows as reviews."
assert len(arg_test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(arg_test_features[:20,:8])


arg_train_labels = np.array([item['label'] for item in arg_data['train'].values()])
arg_test_labels = np.array([item['label'] for item in arg_data['test'].values()])

print(arg_test_labels[:20])

split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(arg_train_features)*split_frac)
arg_train_x, arg_valid_x = arg_train_features[:split_idx], arg_train_features[split_idx:]
arg_train_y, arg_valid_y = arg_train_labels[:split_idx], arg_train_labels[split_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(arg_train_x.shape), 
      "\nValidation set: \t{}".format(arg_valid_x.shape),
      "\nTest set: \t\t{}".format(arg_test_features.shape))

# create Tensor datasets
arg_train_data = TensorDataset(torch.from_numpy(arg_train_x), torch.from_numpy(arg_train_y))
arg_valid_data = TensorDataset(torch.from_numpy(arg_valid_x), torch.from_numpy(arg_valid_y))
arg_test_data = TensorDataset(torch.from_numpy(arg_test_features), torch.from_numpy(arg_test_labels))

# dataloaders
batch_size = 4

# shuffling and batching data
arg_train_loader = DataLoader(arg_train_data, shuffle=True, batch_size=batch_size)
arg_valid_loader = DataLoader(arg_valid_data, shuffle=True, batch_size=batch_size)
arg_test_loader = DataLoader(arg_test_data, shuffle=True, batch_size=batch_size)

In [2]:
# print some example review/sentiment text
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

---
## Data Pre-processing

The first step, when building a neural network, is getting the data into the proper form to feed into the network. Since I'm planning to use a word-embedding layer, I know that I'll need to encode each word in a reviews as an integer, and encode each sentiment label as 1 (positive) or 0 (negative). 

I'll first want to clean up the reviews by removing punctuation and converting them to lowercase. You can see an example of the reviews data, above. Here are the processing steps, I'll want to take:
>* Get rid of any extraneous punctuation.
* You might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string to get all of my text data.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [3]:
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')

all_text = ' '.join(reviews_split)

# create a list of all words
all_words = all_text.split()


### Encoding the Labels

The review labels are "positive" or "negative". To use these labels in a neural network, I need to convert them to numerical values, 1 (positive) and 0 (negative).

In [4]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

### Removing Outliers

As an additional pre-processing step, I want to make sure that the reviews are in good shape for standard processing. That is, I'll want to shape the reviews into a specific, consistent length for ease of processing and comparison. I'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

Before I pad the review text, below, I am checking for reviews of extremely short or long lengths; outliers that may mess with training.

In [5]:
from collections import Counter

# Build a dictionary that maps indices to review lengths
counts = Counter(all_words)

# outlier review stats
# counting words in each review
review_lens = Counter([len(x.split()) for x in reviews_split])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. I seem to have one review with zero length. And, the maximum review length is really long. I'm going to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

In [6]:
print('Number of reviews before removing outliers: ', len(reviews_split))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_split) if len(review.split()) != 0]

# remove 0-length reviews and their labels
reviews_split = [reviews_split[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_split))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


---
## Using a Pre-Trained Embedding Layer

Next, I'll want to tokenize my reviews; turning the list of words that make up a given review into a list of tokenized integers that represent those words. Typically, this is done by creating a dictionary that maps each unique word in a vocabulary to a specific integer value.

In this example, I'll actually want to use a mapping that already exists, in a pre-trained embedding layer. Below, I am loading in a pre-trained embedding model, and I'll explore its traits.

> This code assumes I have a downloaded model `GoogleNews-vectors-negative300-SLIM.bin.gz` in the same directory as this notebook, in a folder, `word2vec_model`.

In [7]:
# # Load a pretrained word2vec model (only need to run code, once)
# ! gzip -d word2vec_model/GoogleNews-vectors-negative300-SLIM.bin.gz

In [3]:
# import Word2Vec loading capabilities
from gensim.models import KeyedVectors

# Creating the model
embed_lookup = KeyedVectors.load_word2vec_format('word2vec_model/GoogleNews-vectors-negative300-SLIM.bin', 
                                                 binary=True)


### Embedding Layer

You can think of an embedding layer as a lookup table, where the rows are indexed by word token and the columns hold the embedding values. For example, row 958 is the embedding vector for the word that maps to the integer value 958.

<img src='notebook_ims/embedding_lookup_table.png' width=40% />

In the below cells, I am storing the words in the pre-trained vocabulary, and printing out the size of the vocabulary and word embeddings. 
> The embedding dimension from the pret-rained model is 300.

In [5]:
# store pretrained vocab
pretrained_words = []
for word in embed_lookup.index_to_key:
    pretrained_words.append(word)
pretrained_words[:5]

['in', 'for', 'that', 'is', 'on']

In [14]:
row_idx = 1

# get word/embedding in that row
word = pretrained_words[row_idx] # get words by index
embedding = embed_lookup[word] # embeddings by word

# vocab and embedding info
print("Size of Vocab: {}\n".format(len(pretrained_words)))
print('Word in vocab: {}\n'.format(word))
print('Length of embedding: {}\n'.format(len(embedding)))
#print('Associated embedding: \n', embedding)

Size of Vocab: 299567

Word in vocab: for

Length of embedding: 300



In [11]:
# print a few common words
for i in range(5):
    print(pretrained_words[i])

in
for
that
is
on


### Cosine Similarity

The pre-trained embedding model has learned to represent semantic relationships between words in vector space. Specifically, words that appear in similar contexts should point in roughly the same direction. To measure whether two vectors are colinear, we can use [**cosine similarity**](https://en.wikipedia.org/wiki/Cosine_similarity), which computes the dot product of two vectors. This dot product is largest when the angle between two vectors is 0 (cos(0) = 1) and cosine is at a maximum, so cosine similarity is larger for aligned vectors.

<img src='notebook_ims/two_vectors.png' width=30% />

### Embedded Bias

Word2Vec, in addition to learning useful similarities and semantic relationships between words, also learns to represent problematic relationships between words. For example, a paper on [Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf) by Bolukbasi et al. (2016), found that the vector-relationship between "man" and "woman" was similar to the relationship between "physician" and "registered nurse" or "shopkeeper" and "housewife" in the trained, Google News Word2Vec model, **which I am using in this notebook**.

>*"In this paper, we quantitatively demonstrate that word-embeddings contain biases in their geometry that reflect gender stereotypes present in broader society. Due to their wide-spread usage as basic
features, word embeddings not only reflect such stereotypes but can also amplify them. This poses a
significant risk and challenge for machine learning and its applications."*

As such, it is important to note that this example is using a Word2Vec model that has been shown to encapsulate gender stereotypes.

You can explore similarities and relationships between word embeddings using code. The code below finds words with the highest cosine similarity when compared to the word `find_similar_to`. 

In [17]:
# Pick a word 
find_similar_to = 'fabulous'

print('Similar words to '+find_similar_to+': \n')

# Find similar words, using cosine similarity
# by default shows top 10 similar words
for similar_word in embed_lookup.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.3f}".format(
        similar_word[0], similar_word[1]
    ))


Similar words to fabulous: 

Word: wonderful, Similarity: 0.761
Word: fantastic, Similarity: 0.761
Word: marvelous, Similarity: 0.730
Word: gorgeous, Similarity: 0.714
Word: lovely, Similarity: 0.713
Word: terrific, Similarity: 0.694
Word: amazing, Similarity: 0.693
Word: beautiful, Similarity: 0.670
Word: magnificent, Similarity: 0.667
Word: splendid, Similarity: 0.645


## Tokenize reviews

The pre-trained embedding layer already has tokens associated with each word in the dictionary. I want to use that same mapping to tokenize all the reviews in the movie review corpus. I will encode any unknown words (words that appear in the reviews but not in the pre-trained vocabulary) as the whitespace token, 0; this should be fine for the purpose of sentiment classification.

In [42]:
# convert reviews to tokens
def tokenize_all_text(embed_lookup, data):
    # split each review into a list of words
    words = [item['text'].split() for item in data.values()]

    tokenized_text = []
    for text in words:
        ints = []
        for word in text:
            try:
                idx = embed_lookup.key_to_index[word]
            except: 
                idx = 0
            ints.append(idx)
        tokenized_text.append(ints)
    
    return tokenized_text


In [43]:
tokenized_text = tokenize_all_text(embed_lookup, data['train'])

In [45]:
test_tokenized_text = tokenize_all_text(embed_lookup, data['test'])

In [179]:
nu_tokenized_text = tokenize_all_text(embed_lookup, nu_data['train'])
nu_test_tokenized_text = tokenize_all_text(embed_lookup, nu_data['test'])

In [180]:
# testing code and printing a tokenized review
print(nu_tokenized_text[0])

[780, 0, 73009, 1, 0, 9560, 0, 116, 798, 192, 9, 194, 3206]


---
## Padding sequences

To deal with both short and very long reviews, I'll pad or truncate all the reviews to a specific length. For reviews shorter than some `seq_length`, I'll left-pad with 0s. For reviews longer than `seq_length`, I'll truncate them to the first `seq_length` words. A good `seq_length`, in this case, is about 200.

> The function `pad_features` returns an array that contains padded, tokenized reviews, of a standard size, that we'll pass to the network. 


As a small example, if the `seq_length=10` and an input, tokenized review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [28]:
def pad_features(tokenized_text, seq_length):
    ''' Return features of tokenized_reviews, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(tokenized_text), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(tokenized_text):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [40]:
# Test your implementation!

seq_length = 50

train_features = pad_features(tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(train_features)==len(tokenized_text), "Features should have as many rows as reviews."
assert len(train_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(train_features[:20,:8])

[[     0    223     49  11534      0      0   4338   5408]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0    897    219      9 130013   1676      4      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      

In [53]:
# Test your implementation!

seq_length = 50

test_features = pad_features(test_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(test_features)==len(test_tokenized_text), "Features should have as many rows as reviews."
assert len(test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(test_features[:20,:8])

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


## NU

In [190]:
# Test your implementation!

seq_length = 15

nu_train_features = pad_features(nu_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(train_features)==len(nu_tokenized_text), "Features should have as many rows as reviews."
assert len(train_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(train_features[:20,:8])

# Test your implementation!

seq_length = 15

nu_test_features = pad_features(nu_test_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(test_features)==len(nu_test_tokenized_text), "Features should have as many rows as reviews."
assert len(test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(test_features[:20,:8])

[[    0     0   780     0 73009     1     0  9560]
 [    0     0     0  4699  4480     9  4516  1207]
 [    0     0     0     0     0     0     0  9660]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0  9660     9 19809     0     0]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0    78     3     9]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0    56]
 [    0     0     0     0  4328 86996  2629     0]
 [    0     0     0     0     0     0  4196     9]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0  7460]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0]
 [    0     0     0     0     0 23690     9   459]
 [  132    53    38  5838     9   389     0  1031]
 [    0     0     0     0     0

---
## Training, Validation, and Test Data

With the data in nice shape, I'll split it into training, validation, and test sets.

In the below code, I am creating features (x) and labels (y). 
* The split fraction, `split_frac` defines the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left is split in half to create the validation and test data.

In [62]:
train_labels = np.array([item['label'] for item in data['train'].values()])
test_labels = np.array([item['label'] for item in data['test'].values()])

In [188]:
#NU
nu_train_labels = np.array([item['label'] for item in nu_data['train'].values()])
nu_test_labels = np.array([item['label'] for item in nu_data['test'].values()])

In [197]:
nu_train_labels[:20]

array([2, 0, 1, 1, 1, 0, 0, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 0, 2, 2])

In [64]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(train_features)*split_frac)
train_x, valid_x = train_features[:split_idx], train_features[split_idx:]
train_y, valid_y = train_labels[:split_idx], train_labels[split_idx:]

In [191]:
#NU
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(nu_train_features)*split_frac)
nu_train_x, nu_valid_x = nu_train_features[:split_idx], nu_train_features[split_idx:]
nu_train_y, nu_valid_y = nu_train_labels[:split_idx], nu_train_labels[split_idx:]


In [192]:
# split_frac = 0.8

# ## split data into training, validation, and test data (features and labels, x and y)

# split_idx = int(len(features)*split_frac)
# train_x, remaining_x = features[:split_idx], features[split_idx:]
# train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

# test_idx = int(len(remaining_x)*0.5)
# val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
# val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]



## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(nu_train_x.shape), 
      "\nValidation set: \t{}".format(nu_valid_x.shape),
      "\nTest set: \t\t{}".format(nu_test_features.shape))

			Feature Shapes:
Train set: 		(333, 15) 
Validation set: 	(84, 15) 
Test set: 		(179, 15)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## DataLoaders and Batching

After creating training, test, and validation data, I can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [151]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_features), torch.from_numpy(test_labels))

# dataloaders
batch_size = 4

# shuffling and batching data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [193]:
# NU
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
nu_train_data = TensorDataset(torch.from_numpy(nu_train_x), torch.from_numpy(nu_train_y))
nu_valid_data = TensorDataset(torch.from_numpy(nu_valid_x), torch.from_numpy(nu_valid_y))
nu_test_data = TensorDataset(torch.from_numpy(nu_test_features), torch.from_numpy(nu_test_labels))

# dataloaders
batch_size = 4

# shuffling and batching data
nu_train_loader = DataLoader(nu_train_data, shuffle=True, batch_size=batch_size)
nu_valid_loader = DataLoader(nu_valid_data, shuffle=True, batch_size=batch_size)
nu_test_loader = DataLoader(nu_test_data, shuffle=True, batch_size=batch_size)

In [204]:
import json
with open("../preprocess/arg_split.json") as f:
    arg_data = json.load(f)

arg_tokenized_text = tokenize_all_text(embed_lookup, arg_data['train'])
arg_test_tokenized_text = tokenize_all_text(embed_lookup, arg_data['test'])

# Test your implementation!

seq_length = 15

arg_train_features = pad_features(arg_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(arg_train_features)==len(arg_tokenized_text), "Features should have as many rows as reviews."
assert len(arg_train_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(arg_train_features[:20,:8])

# Test your implementation!

arg_test_features = pad_features(arg_test_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(arg_test_features)==len(arg_test_tokenized_text), "Features should have as many rows as reviews."
assert len(arg_test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(arg_test_features[:20,:8])


arg_train_labels = np.array([item['label'] for item in arg_data['train'].values()])
arg_test_labels = np.array([item['label'] for item in arg_data['test'].values()])

print(arg_test_labels[:20])

split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(arg_train_features)*split_frac)
arg_train_x, arg_valid_x = arg_train_features[:split_idx], arg_train_features[split_idx:]
arg_train_y, arg_valid_y = arg_train_labels[:split_idx], arg_train_labels[split_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(arg_train_x.shape), 
      "\nValidation set: \t{}".format(arg_valid_x.shape),
      "\nTest set: \t\t{}".format(arg_test_features.shape))

# create Tensor datasets
arg_train_data = TensorDataset(torch.from_numpy(arg_train_x), torch.from_numpy(arg_train_y))
arg_valid_data = TensorDataset(torch.from_numpy(arg_valid_x), torch.from_numpy(arg_valid_y))
arg_test_data = TensorDataset(torch.from_numpy(arg_test_features), torch.from_numpy(arg_test_labels))

# dataloaders
batch_size = 4

# shuffling and batching data
arg_train_loader = DataLoader(arg_train_data, shuffle=True, batch_size=batch_size)
arg_valid_loader = DataLoader(arg_valid_data, shuffle=True, batch_size=batch_size)
arg_test_loader = DataLoader(arg_test_data, shuffle=True, batch_size=batch_size)

[[   986      9   2887     12    390  58549     22     37]
 [     0     25  22694      9    770      0     78  72206]
 [     0  15370  10297  15760   1504      0   1444      0]
 [     9  24325   1947      8   2041      0      9    271]
 [     9   6398      2   6837     78   1599      0    192]
 [  5097      0   4408      0   1015      0  11542   3677]
 [    40   4932      0   1218   2000     22     37      2]
 [    34    857      0      9   4588      0      0  13080]
 [  4818     35   1471   5695      0   9577  21774      9]
 [    66   3411      3      0   8561      0 231574    179]
 [    34   2093   3980      0   7712      0  60560  10492]
 [    25   1427   2000  22915 104856 231574      1  36464]
 [  1682     24   7558   7110      0    129      0    125]
 [     0      0      0     34   4094      0     60   5426]
 [     0      0      9   8190    483     16      9   1931]
 [     0    957      9   8521   3936      0  80045  37286]
 [     0   5526   2878   3106    160   3079      0   210

In [214]:
# LREC
import json
with open("../preprocess/lrec_split.json") as f:
    lrec_data = json.load(f)

lrec_tokenized_text = tokenize_all_text(embed_lookup, lrec_data['train'])
lrec_test_tokenized_text = tokenize_all_text(embed_lookup, lrec_data['test'])

# Test your implementation!

seq_length = 15

lrec_train_features = pad_features(lrec_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(lrec_train_features)==len(lrec_tokenized_text), "Features should have as many rows as reviews."
assert len(lrec_train_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(lrec_train_features[:20,:8])

# Test your implementation!

lrec_test_features = pad_features(lrec_test_tokenized_text, seq_length=seq_length)

## test statements - do not change - ##
assert len(lrec_test_features)==len(lrec_test_tokenized_text), "Features should have as many rows as reviews."
assert len(lrec_test_features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 8 values of the first 20 batches 
print(lrec_test_features[:20,:8])


lrec_train_labels = np.array([item['label'] for item in lrec_data['train'].values()])
lrec_test_labels = np.array([item['label'] for item in lrec_data['test'].values()])

print(lrec_test_labels[:20])

split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(lrec_train_features)*split_frac)
lrec_train_x, lrec_valid_x = lrec_train_features[:split_idx], lrec_train_features[split_idx:]
lrec_train_y, lrec_valid_y = lrec_train_labels[:split_idx], lrec_train_labels[split_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(lrec_train_x.shape), 
      "\nValidation set: \t{}".format(lrec_valid_x.shape),
      "\nTest set: \t\t{}".format(lrec_test_features.shape))

# create Tensor datasets
lrec_train_data = TensorDataset(torch.from_numpy(lrec_train_x), torch.from_numpy(lrec_train_y))
lrec_valid_data = TensorDataset(torch.from_numpy(lrec_valid_x), torch.from_numpy(lrec_valid_y))
lrec_test_data = TensorDataset(torch.from_numpy(lrec_test_features), torch.from_numpy(lrec_test_labels))

# dataloaders
batch_size = 4

# shuffling and batching data
lrec_train_loader = DataLoader(lrec_train_data, shuffle=True, batch_size=batch_size)
lrec_valid_loader = DataLoader(lrec_valid_data, shuffle=True, batch_size=batch_size)
lrec_test_loader = DataLoader(lrec_test_data, shuffle=True, batch_size=batch_size)

[[     0      0      0      0      0      0  31722     78]
 [     0      0      0      0      0    132      3      9]
 [     0      0      0      0      0      0      0      0]
 [128719    814      0    134      4      9   2032   8748]
 [     0      0      0      0      0      0      0     78]
 [    83     34   6510      9   3042      0      9 137375]
 [   132    330     90     45     34   9593    545   8229]
 [    50   1748      9    708      4   1172  17670  77415]
 [     0      0      0      0      0      0      0   2873]
 [     0      0      0   1715   2962      0    245    116]
 [     0      0      0      0      0      0      0      0]
 [  4779     78     21   1090     56   4365   1183      9]
 [     0      0      0      0     78     83      9   6149]
 [   132    171      9   9519      0      9    527   5129]
 [   494     12    124   3887     12     38     45      2]
 [     0      0      0     78      3      9   1840      0]
 [  4779     78     21   1090      0      9  12761     5

------
# Bi-LSTM


In [251]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BiLSTMSentiment(nn.Module):

    def __init__(self, embed_model, vocab_size, label_size, embedding_dim, hidden_dim, batch_size, seq_length, dropout=0.5, freeze_embeddings = True):
        super(BiLSTMSentiment, self).__init__()
        # set class vars
        self.embedding_dim = embedding_dim
        
        # 1. embedding layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embeddings.weight = nn.Parameter(torch.from_numpy(embed_model.vectors)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embeddings.requires_grad = False

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.dropout = dropout

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(batch_first=True, input_size=embedding_dim, num_layers =2, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2, label_size)

    def forward(self, sentence):
        x = self.embeddings(sentence)
        _, (lstm_out, _) = self.lstm(x)
        logits = self.hidden2label(lstm_out[-1])
        # log_probs = F.log_softmax(logits)
        return logits

In [ ]:
# Get test data loss and accuracy
# ARg

test_losses = [] # track loss
num_correct = 0
pred_tensor = None
label_tensor = None


arg_net.eval()
# iterate over test data
for inputs, labels in arg_test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = arg_net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.argmax(output, dim=1)  # argmax

    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    if pred_tensor == None:
        pred_tensor = pred
    else:
        pred_tensor = torch.cat((pred_tensor, pred), dim=-1)

    if label_tensor == None:
        label_tensor = labels
    else:
        label_tensor = torch.cat((label_tensor, labels), dim=-1)

from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall

print("micro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4).item()))
print("macro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted F1: {:.3f}\n".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))

print("micro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4).item()))
print("macro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted precision: {:.3f}\n".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))

print("micro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4).item()))
print("macro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted recall: {:.3f}\n".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))


micro F1: 0.439
macro F1: 0.208
weighted F1: 0.427

micro precision: 0.439
macro precision: 0.249
weighted precision: 0.465

micro recall: 0.439
macro recall: 0.225
weighted recall: 0.439



---
# Sentiment Network with PyTorch

The complete model is made of a few layers:

**1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding)**
* This converts our word tokens (integers) into embedded vectors of a specific size.
* In this case, the vectors/weights of this layer will come from a **pretrained** lookup table. 

**2. A few [convolutional layers](https://pytorch.org/docs/stable/nn.html#conv1d)**
* These are defined by an input size, number of filters/feature maps to output, and a kernel size.
* The output of these layers will go through a ReLu activation function and pooling layer in the `forward` function.

**3. A fully-connected, output layer**
* This maps the convolutional layer outputs to a desired output_size (1 sentiment class).

**4. A sigmoid activation layer**
* This turns the output logit into a value 0-1; a class score.

There is also a dropout layer, which will prevent overfitting, placed between the convolutional outputs and the final, fully-connected layer.

<img src="notebook_ims/complete_embedding_CNN.png" width=60%>

*Image from the original paper, [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/pdf/1408.5882.pdf).*

### The Embedding Layer

The embedding layer comes from our pre-trained `embed_lookup` model. By default, the weights of this layer are set to the vectors from the pre-trained model and frozen, so it will just be used as a lookup table. You could train your own embedding layer here, but it will speed up the training process to use a pre-trained model.

### The Convolutional Layer(s)

I am creating three convolutional layers, which will have kernel_sizes of (3, 300), (4, 300), and (5, 300); to look at 3-, 4-, and 5- sequences of word embeddings at a time. Each of these three layers will produce  100 filtered outputs. This is following the layer conventions in the paper, [CNNs for Sentence Classification](https://arxiv.org/abs/1408.5882).

> The kernels only move in one dimension: down a sequence of word embeddings. In other words, these kernels move along a sequence of words, in time!

### Maxpooling Layers

In the `forward` function, I am applying a ReLu activation to the outputs of all convolutional layers and a maxpooling layer over the input sequence dimension. The maxpooling layer will get us an indication of whether some high-level text feature was found. 

> After moving through 3 convolutional layers with 100 filtered outputs each, these layers will output 300 values that can be sent to a final, fully-connected, classification layer.

In [152]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [194]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self, embed_model, vocab_size, output_size, embedding_dim,
                 num_filters=100, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentCNN, self).__init__()

        # set class vars
        self.num_filters = num_filters
        self.embedding_dim = embedding_dim
        
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(embed_model.vectors)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False
        
        # 2. convolutional layers
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim), padding=(k-2,0)) 
            for k in kernel_sizes])
        
        # 3. final, fully-connected layer for classification
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_size) 
        
        # 4. dropout and sigmoid layers
        self.dropout = nn.Dropout(drop_prob)
        # self.sig = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    
    def conv_and_pool(self, x, conv):
        """
        Convolutional + max pooling layer
        """
        # squeeze last dim to get size: (batch_size, num_filters, conv_seq_length)
        # conv_seq_length will be ~ 200
        x = F.relu(conv(x)).squeeze(3)
        
        # 1D pool over conv_seq_length
        # squeeze to get size: (batch_size, num_filters)
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        """
        Defines how a batch of inputs, x, passes through the model layers.
        Returns a single, sigmoid-activated class score as output.
        """
        # embedded vectors
        embeds = self.embedding(x) # (batch_size, seq_length, embedding_dim)
        # embeds.unsqueeze(1) creates a channel dimension that conv layers expect
        embeds = embeds.unsqueeze(1)
        
        # get output of each conv-pool layer
        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]
        
        # concatenate results and add dropout
        x = torch.cat(conv_results, 1)
        x = self.dropout(x)
        
        # final logit
        logit = self.fc(x) 
        
        # raw logits for CE loss
        return logit
      

## Instantiate the network

Here, I'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `num_filters`: Number of filters that each convolutional layer produces as output.
* `filter_sizes`: A list of kernel sizes; one convolutional layer will be created for each kernel size.

Any parameters I did not list, are left as the default value.

In [154]:
# Instantiate the model w/ hyperparams

vocab_size = len(pretrained_words)
output_size = 3 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(net)

SentimentCNN(
  (embedding): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)


In [233]:
# arc lstm

vocab_size = len(pretrained_words)
output_size = 3 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors

hidden_dim = 128

batch_size = 4

arc_lstm = BiLSTMSentiment(embed_lookup, vocab_size, output_size, embedding_dim,
                   hidden_dim, batch_size)

print(arc_lstm)

BiLSTMSentiment(
  (embeddings): Embedding(299567, 300)
  (lstm): LSTM(300, 128, bidirectional=True)
  (hidden2label): Linear(in_features=256, out_features=3, bias=True)
)


In [195]:
# Instantiate the model w/ hyperparams
#NU

vocab_size = len(pretrained_words)
output_size = 3 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

nu_net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(nu_net)

SentimentCNN(
  (embedding): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)


In [208]:
# Instantiate the model w/ hyperparams
# ARG

vocab_size = len(pretrained_words)
output_size = 4 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

arg_net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(arg_net)

SentimentCNN(
  (embedding): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)


In [215]:
# Instantiate the model w/ hyperparams
# lrec

vocab_size = len(pretrained_words)
output_size = 3 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

lrec_net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(lrec_net)

SentimentCNN(
  (embedding): Embedding(299567, 300)
  (convs_1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (fc): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (softmax): Softmax(dim=1)
)


In [252]:
# Instantiate the model w/ hyperparams


# lrec lstm

vocab_size = len(pretrained_words)
output_size = 3 # binary class (1 or 0)
embedding_dim = len(embed_lookup[pretrained_words[0]]) # 300-dim vectors

hidden_dim = 128

batch_size = 4
seq_length = 15

lrec_lstm = BiLSTMSentiment(embed_lookup, vocab_size, output_size, embedding_dim,
                   hidden_dim, batch_size, seq_length)

print(lrec_lstm)

BiLSTMSentiment(
  (embeddings): Embedding(299567, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, bidirectional=True)
  (hidden2label): Linear(in_features=256, out_features=3, bias=True)
)


---
## Training

Below is some training code, which iterates over all of the training data, records some loss statistics and performs backpropagation + optimization steps to update the weights of this network.

>I'll also be using a binary cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

I also have some training hyperparameters:

* `lr`: Learning rate for the optimizer.
* `epochs`: Number of times to iterate through the training dataset.

In [148]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay = 0.01)


In [235]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
nu_optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay = 0.01)

In [155]:
# training loop
def train(net, train_loader, epochs, print_every=100):

    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    counter = 0 # for printing
    
    # train for some number of epochs
    net.train()
    for e in range(epochs):

        # batch loop
        for inputs, labels in train_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output = net(inputs)

            # calculate the loss and perform backprop
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_losses = []
                net.eval()
                for inputs, labels in valid_loader:

                    if(train_on_gpu):
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output = net(inputs)
                    val_loss = criterion(output, labels)

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [156]:
# training params

epochs = 5 # this is approx where I noticed the validation loss stop decreasing
print_every = 10

train(net, train_loader, epochs, print_every=print_every)

Epoch: 1/5... Step: 10... Loss: 1.121224... Val Loss: 1.093208
Epoch: 1/5... Step: 20... Loss: 1.101917... Val Loss: 1.093345
Epoch: 1/5... Step: 30... Loss: 1.097594... Val Loss: 1.093285
Epoch: 1/5... Step: 40... Loss: 1.127493... Val Loss: 1.093037
Epoch: 2/5... Step: 50... Loss: 1.111258... Val Loss: 1.092817
Epoch: 2/5... Step: 60... Loss: 1.033905... Val Loss: 1.092382
Epoch: 2/5... Step: 70... Loss: 1.074409... Val Loss: 1.091868
Epoch: 2/5... Step: 80... Loss: 0.989854... Val Loss: 1.091301
Epoch: 3/5... Step: 90... Loss: 1.064365... Val Loss: 1.090846
Epoch: 3/5... Step: 100... Loss: 1.086898... Val Loss: 1.090428
Epoch: 3/5... Step: 110... Loss: 1.144946... Val Loss: 1.090008
Epoch: 3/5... Step: 120... Loss: 1.170827... Val Loss: 1.089649
Epoch: 4/5... Step: 130... Loss: 1.039327... Val Loss: 1.089281
Epoch: 4/5... Step: 140... Loss: 1.099073... Val Loss: 1.088922
Epoch: 4/5... Step: 150... Loss: 1.140197... Val Loss: 1.088559
Epoch: 4/5... Step: 160... Loss: 0.982836... Val 

In [200]:
# training params

epochs = 15 # this is approx where I noticed the validation loss stop decreasing
print_every = 10

train(nu_net, train_loader, epochs, print_every=print_every)

Epoch: 1/15... Step: 10... Loss: 1.119029... Val Loss: 1.076940
Epoch: 1/15... Step: 20... Loss: 0.976671... Val Loss: 1.076753
Epoch: 1/15... Step: 30... Loss: 1.172690... Val Loss: 1.076578
Epoch: 1/15... Step: 40... Loss: 1.287001... Val Loss: 1.076412
Epoch: 2/15... Step: 50... Loss: 1.029759... Val Loss: 1.076251
Epoch: 2/15... Step: 60... Loss: 1.093583... Val Loss: 1.076088
Epoch: 2/15... Step: 70... Loss: 1.196053... Val Loss: 1.075928
Epoch: 2/15... Step: 80... Loss: 1.058536... Val Loss: 1.075770
Epoch: 3/15... Step: 90... Loss: 1.221428... Val Loss: 1.075621
Epoch: 3/15... Step: 100... Loss: 1.163355... Val Loss: 1.075482
Epoch: 3/15... Step: 110... Loss: 1.364627... Val Loss: 1.075346
Epoch: 3/15... Step: 120... Loss: 0.895639... Val Loss: 1.075215
Epoch: 4/15... Step: 130... Loss: 1.091024... Val Loss: 1.075088
Epoch: 4/15... Step: 140... Loss: 1.095452... Val Loss: 1.074970
Epoch: 4/15... Step: 150... Loss: 1.069494... Val Loss: 1.074861
Epoch: 4/15... Step: 160... Loss: 

In [206]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay = 0.01)

In [212]:
# training params

epochs = 5 # this is approx where I noticed the validation loss stop decreasing
print_every = 10

train(arg_net, arg_train_loader, epochs, print_every=print_every)

Epoch: 1/5... Step: 10... Loss: 1.367653... Val Loss: 1.404443
Epoch: 1/5... Step: 20... Loss: 1.342993... Val Loss: 1.403332
Epoch: 1/5... Step: 30... Loss: 1.432281... Val Loss: 1.402293
Epoch: 1/5... Step: 40... Loss: 1.332261... Val Loss: 1.401276
Epoch: 1/5... Step: 50... Loss: 1.404521... Val Loss: 1.400265
Epoch: 1/5... Step: 60... Loss: 1.459815... Val Loss: 1.399292
Epoch: 1/5... Step: 70... Loss: 1.376378... Val Loss: 1.398344
Epoch: 1/5... Step: 80... Loss: 1.439578... Val Loss: 1.397355
Epoch: 1/5... Step: 90... Loss: 1.432746... Val Loss: 1.396285
Epoch: 1/5... Step: 100... Loss: 1.471794... Val Loss: 1.395255
Epoch: 1/5... Step: 110... Loss: 1.334440... Val Loss: 1.394261
Epoch: 1/5... Step: 120... Loss: 1.521729... Val Loss: 1.393342
Epoch: 1/5... Step: 130... Loss: 1.373954... Val Loss: 1.392389
Epoch: 1/5... Step: 140... Loss: 1.453811... Val Loss: 1.391366
Epoch: 1/5... Step: 150... Loss: 1.338229... Val Loss: 1.390343
Epoch: 1/5... Step: 160... Loss: 1.364637... Val 

In [222]:
# training params

epochs = 2 # this is approx where I noticed the validation loss stop decreasing
print_every = 10

train(lrec_net, lrec_train_loader, epochs, print_every=print_every)

Epoch: 1/2... Step: 10... Loss: 1.029313... Val Loss: 1.078715
Epoch: 1/2... Step: 20... Loss: 1.090346... Val Loss: 1.078658
Epoch: 1/2... Step: 30... Loss: 0.899229... Val Loss: 1.078598
Epoch: 1/2... Step: 40... Loss: 1.137216... Val Loss: 1.078538
Epoch: 2/2... Step: 50... Loss: 1.202685... Val Loss: 1.078482
Epoch: 2/2... Step: 60... Loss: 1.170326... Val Loss: 1.078425
Epoch: 2/2... Step: 70... Loss: 1.025374... Val Loss: 1.078367
Epoch: 2/2... Step: 80... Loss: 1.144756... Val Loss: 1.078319
Epoch: 2/2... Step: 90... Loss: 1.121153... Val Loss: 1.078274


In [253]:
# training params

epochs = 2 # this is approx where I noticed the validation loss stop decreasing
print_every = 10

train(lrec_lstm, lrec_train_loader, epochs, print_every=print_every)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x128 and 256x3)

---
## Testing

There are a few ways to test this network.

* **Test data performance:** First, I'll see how our trained model performs on all of the defined test_data, above; I'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, I'll see if I can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [163]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0
pred_tensor = None
label_tensor = None


net.eval()
# iterate over test data
for inputs, labels in test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.argmax(output, dim=1)  # argmax

    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    if pred_tensor == None:
        pred_tensor = pred
    else:
        pred_tensor = torch.cat((pred_tensor, pred), dim=-1)

    if label_tensor == None:
        label_tensor = labels
    else:
        label_tensor = torch.cat((label_tensor, labels), dim=-1)



# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))



Test loss: 1.070
Test accuracy: 0.512


In [164]:
label_tensor

tensor([1, 2, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 2, 2, 1, 2, 0, 1, 0, 0,
        0, 1, 2, 1, 2, 2, 1, 1, 1, 0, 0, 1, 2, 1, 2, 1, 1, 2, 0, 1, 0, 1, 2, 1,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 1])

In [165]:
pred_tensor

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [176]:
# ARC
from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall

print("micro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3).item()))
print("macro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted F1: {:.3f}\n".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3).item()))
print("macro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted precision: {:.3f}\n".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3).item()))
print("macro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted recall: {:.3f}\n".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))


micro F1: 0.512
macro F1: 0.226
weighted F1: 0.347

micro precision: 0.512
macro precision: 0.171
weighted precision: 0.262

micro recall: 0.512
macro recall: 0.333
weighted recall: 0.512



In [203]:
# Get test data loss and accuracy
# NU

test_losses = [] # track loss
num_correct = 0
pred_tensor = None
label_tensor = None


nu_net.eval()
# iterate over test data
for inputs, labels in nu_test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = nu_net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.argmax(output, dim=1)  # argmax

    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    if pred_tensor == None:
        pred_tensor = pred
    else:
        pred_tensor = torch.cat((pred_tensor, pred), dim=-1)

    if label_tensor == None:
        label_tensor = labels
    else:
        label_tensor = torch.cat((label_tensor, labels), dim=-1)

from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall

print("micro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3).item()))
print("macro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted F1: {:.3f}\n".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3).item()))
print("macro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted precision: {:.3f}\n".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3).item()))
print("macro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted recall: {:.3f}\n".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

# # -- stats! -- ##
# # avg test loss
# print("Test loss: {:.3f}".format(np.mean(test_losses)))

# # accuracy over all test data
# test_acc = num_correct/len(test_loader.dataset)
# print("Test accuracy: {:.3f}".format(test_acc))





micro F1: 0.145
macro F1: 0.085
weighted F1: 0.037

micro precision: 0.145
macro precision: 0.048
weighted precision: 0.021

micro recall: 0.145
macro recall: 0.333
weighted recall: 0.145



In [213]:
# Get test data loss and accuracy
# ARg

test_losses = [] # track loss
num_correct = 0
pred_tensor = None
label_tensor = None


arg_net.eval()
# iterate over test data
for inputs, labels in arg_test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = arg_net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.argmax(output, dim=1)  # argmax

    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    if pred_tensor == None:
        pred_tensor = pred
    else:
        pred_tensor = torch.cat((pred_tensor, pred), dim=-1)

    if label_tensor == None:
        label_tensor = labels
    else:
        label_tensor = torch.cat((label_tensor, labels), dim=-1)

from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall

print("micro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4).item()))
print("macro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted F1: {:.3f}\n".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))

print("micro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4).item()))
print("macro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted precision: {:.3f}\n".format(multiclass_precision(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))

print("micro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4).item()))
print("macro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4, average='macro').item()))
print("weighted recall: {:.3f}\n".format(multiclass_recall(pred_tensor, label_tensor, num_classes=4, average='weighted').item()))


micro F1: 0.439
macro F1: 0.208
weighted F1: 0.427

micro precision: 0.439
macro precision: 0.249
weighted precision: 0.465

micro recall: 0.439
macro recall: 0.225
weighted recall: 0.439



In [223]:
# Get test data loss and accuracy
# lrec

test_losses = [] # track loss
num_correct = 0
pred_tensor = None
label_tensor = None


lrec_net.eval()
# iterate over test data
for inputs, labels in lrec_test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = lrec_net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.argmax(output, dim=1)  # argmax

    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    if pred_tensor == None:
        pred_tensor = pred
    else:
        pred_tensor = torch.cat((pred_tensor, pred), dim=-1)

    if label_tensor == None:
        label_tensor = labels
    else:
        label_tensor = torch.cat((label_tensor, labels), dim=-1)

from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics.functional import multiclass_precision
from torcheval.metrics.functional import multiclass_recall

print("micro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3).item()))
print("macro F1: {:.3f}".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted F1: {:.3f}\n".format(multiclass_f1_score(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3).item()))
print("macro precision: {:.3f}".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted precision: {:.3f}\n".format(multiclass_precision(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))

print("micro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3).item()))
print("macro recall: {:.3f}".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='macro').item()))
print("weighted recall: {:.3f}\n".format(multiclass_recall(pred_tensor, label_tensor, num_classes=3, average='weighted').item()))


micro F1: 0.423
macro F1: 0.198
weighted F1: 0.252

micro precision: 0.423
macro precision: 0.141
weighted precision: 0.179

micro recall: 0.423
macro recall: 0.333
weighted recall: 0.423



### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!

> The below `predict` code, takes in a trained `embed_lookup` table, a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!


In [27]:
from string import punctuation

# helper function to process and tokenize a single review
def tokenize_review(embed_lookup, test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    tokenized_review = []
    for word in test_words:
        try:
            idx = embed_lookup.vocab[word].index
        except: 
            idx = 0
        tokenized_review.append(idx)

    return tokenized_review


In [28]:
def predict(embed_lookup, net, test_review, sequence_length=200):
    """
    Predict whether a given test_review has negative or positive sentiment.
    """
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(embed_lookup, test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features([test_ints], seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output = net(feature_tensor)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")
        

### Test on pos/neg reviews

Below, I test my code on both positive and negative reviews.

In [29]:
# set hyperparams
seq_length=200 # good to use the length that was trained on


In [30]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

# test negative review
predict(embed_lookup, net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.000775
Negative review detected.


In [31]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

predict(embed_lookup, net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.992333
Positive review detected!


## Try out test reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment.

---
## Further reading

More than text classification, CNNs are used to analyze sequential data in a number of ways! Here are a couple of papers and applications that I find really interesting:
* CNN for semantic representations and **search query retrieval**, [paper (Microsoft)](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/www2014_cdssm_p07.pdf).
* CNN for **genetic mutation detection**, [paper (Nature)](https://www.nature.com/articles/s41467-019-09027-x).
* CNN for classifying [whale sounds](https://ai.googleblog.com/2018/10/acoustic-detection-of-humpback-whales.html) via spectogram and for [**audio classification**, generally (Google AI)](https://ai.google/research/pubs/pub45611).